In [25]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from random import seed
from skimage.io import imread,imshow
from skimage.transform import resize
from tqdm import tqdm
import tensorflow as tf
import tensorboard
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K

# Jaccard Coefficients

In [26]:
def jaccard_coef(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f*y_pred_f)
    output = (intersection + 1.0)/(K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)
    return output


def jaccard_coef_loss(y_true,y_pred):
    return -jaccard_coef(y_true,y_pred)

# U-NET Architecture

In [27]:
def conv2d_block(input_tensor, n_filters, dropout=0.1,kernel_size=3,batchnorm=True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    #first layer
    x = Conv2D(filters = n_filters, kernel_size= (kernel_size,kernel_size),
               kernel_initializer = 'he_normal',padding='same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #Middle Layer
    x = Dropout(dropout)(x)
    
    #second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size,kernel_size),
              kernel_initializer='he_normal',padding='same')(input_tensor)
    
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x


In [28]:
def get_unet(input_img,n_filters = 16,batchnorm = True):
    
    #Contracting Path
    c1 = conv2d_block(input_img,n_filters*1,kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2,2))(c1)
    
    c2 = conv2d_block(p1,n_filters*2,kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2,2))(c2)
    
    c3 = conv2d_block(p2,n_filters*4,kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2,2))(c3)
    
    c4 = conv2d_block(p3,n_filters*8,dropout=0.2,kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D((2,2))(c4)

    c5 = conv2d_block(p4,n_filters=n_filters*16,dropout=0.3, kernel_size=3, batchnorm=batchnorm)

    #Expansive path
    u6 = Conv2DTranspose(n_filters*8,(3,3),strides= (2,2),padding='same')(c5)
    u6 = concatenate([u6,c4])
    c6 = conv2d_block(u6,n_filters*8,kernel_size=3,batchnorm=batchnorm)
    
    u7 = Conv2DTranspose(n_filters*4,(3,3),strides= (2,2),padding='same')(c6)
    u7 = concatenate([u7,c3])
    c7 = conv2d_block(u7,n_filters*4,kernel_size=3,batchnorm=batchnorm)
    
    u8 = Conv2DTranspose(n_filters*2,(3,3),strides= (2,2),padding='same')(c7)
    u8 = concatenate([u8,c2])
    c8 = conv2d_block(u8,n_filters*4,kernel_size=3,batchnorm=batchnorm)
    
    u9 = Conv2DTranspose(n_filters*1,(3,3),strides= (2,2),padding='same')(c8)
    u9 = concatenate([u9,c1])
    c9 = conv2d_block(u9,n_filters*1,kernel_size=3,batchnorm=batchnorm)

    outputs = Conv2D(1,(1,1),activation='sigmoid')(c9)
    model = Model(inputs=[input_img],outputs=[outputs])
    return model


In [29]:
def unet_model_with_jaccard(IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS):
    
    inputs = Input((IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))
    input_image = tf.keras.layers.Lambda(lambda x:x/255)(inputs)
    model = get_unet(input_image,n_filters=16,batchnorm=True)
    model.compile(optimizer='adam',loss=[jaccard_coef_loss],metrics=[jaccard_coef])
    model.summary()
    return model